# products.py Module

Handles product CRUD, filtering, and search logic.

# Products Module
All public product listing, filtering, and related logic from the Flask app.

In [ ]:
from flask import request, render_template
from contextlib import closing
import re

# Import get_db from the db module (already loaded by app.py)
from db import get_db

# --- Public Products Listing ---
def get_products_page(request):
    """
    Main function called by app.py for the /products route
    """
    search_query = request.args.get('q', '').strip()
    filter_type = request.args.get('filter', 'all')
    zip_param = (request.args.get('zip', '') or '').strip()
    
    with closing(get_db()) as db:
        cur = db.cursor()
        base_query = (
            "SELECT p.id, p.title, p.description, p.price, p.provider_id, "
            "prov.business_name as provider_business_name, prov.base_zip as provider_base_zip "
            "FROM products p "
            "LEFT JOIN providers prov ON p.provider_id = prov.id "
            "WHERE p.active = 1"
        )
        params = []
        
        if search_query:
            base_query += " AND (LOWER(p.title) LIKE LOWER(?) OR LOWER(p.description) LIKE LOWER(?) OR LOWER(prov.business_name) LIKE LOWER(?) OR LOWER(prov.first_name) LIKE LOWER(?))"
            search_param = f"%{search_query}%"
            params.extend([search_param, search_param, search_param, search_param])
        
        if filter_type == 'best_price':
            base_query += " ORDER BY CASE WHEN p.price LIKE '$%' THEN CAST(SUBSTR(p.price, 2, INSTR(p.price || '-', '-') - 2) AS INTEGER) ELSE 999999 END ASC, p.created_at DESC"
        elif filter_type == 'by_provider':
            base_query += " ORDER BY COALESCE(prov.business_name, 'Independent Contributor'), p.title ASC"
        else:
            base_query += " ORDER BY p.created_at DESC"
        
        cur.execute(base_query, params)
        products_raw = cur.fetchall()
        
        # Convert rows to dictionaries
        products_list = [
            {
                'id': row['id'],
                'title': row['title'],
                'description': row['description'],
                'price': row['price'],
                'provider_id': row['provider_id'],
                'provider_business_name': row['provider_business_name']
            }
            for row in products_raw
        ]
        
        # Get profile for template context
        cur.execute("SELECT first_name, business_name FROM profile WHERE id = 1")
        profile_row = cur.fetchone()
        profile = {
            "first_name": profile_row["first_name"] if profile_row else "",
            "business_name": profile_row["business_name"] if profile_row else "Your Service Provider"
        }
    
    return render_template(
        "products.html",
        products=products_list,
        profile=profile,
        search_query=search_query,
        filter_type=filter_type,
        title="All Products",
    )
